# EM

In [1]:
import numpy as np
import math
from scipy.special import logsumexp
from numpy import linalg as LA

In [99]:
# Load data
X = np.load("./X.npy")

In [100]:
X

array([[2, 2, 1, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 2, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 2, 1, 1]])

In [101]:
n = X.shape[0]
d = X.shape[1]

In [150]:
# Initialization
a = np.array([0.5, 0.4, 0.1])
b = np.array([0.5, 0.4, 0.1])
c = np.array([0.5, 0.4, 0.1])
p = np.array([float(1)/(d-3)] * (d-3))

In [103]:
# Function to compute the wik matrix, as specified in question (d)
def compute_w(X, a, b, c, p):
    v = np.empty([n, d-3])
    for j in range(d-3):
        v[:,j] = np.log(a[X[:, j]]) + np.log(a[X[:,j+1]]) + np.log(b[X[:, j+2]]) + np.log(b[X[:, j+3]]) \
                + np.log(p[j]) + np.sum(np.array([np.log(c[X[:,p]]) for p in range(d) if p not in range(j,j+4)]), axis = 0)
            
    dd = logsumexp(v, axis = 1)
    w = np.exp(v-dd[:,None])
    return w

In [149]:
# Discussed with Zihao Wang
# Function to update parameters as specified in question (d)
def update_parameters(X, w,a,b,c,p):
    theta_old = np.concatenate((a,b,c))
    
    p_new = w.mean(axis = 0)

    a_m = np.zeros([n,d-3,3])
    b_m = np.zeros([n,d-3,3])
    c_m = np.zeros([n,d-3,3])
    
    for i in range(3):
        for k in range(d-3):
            a_m[:,k,i] = (X[:,k:k+2] == i).sum(axis = 1) * w[:,k]
            b_m[:,k,i] = (X[:,k+2:k+4] == i).sum(axis = 1) * w[:,k]
            c_m[:,k,i] = ((X[:,:k] == i).sum(axis = 1)+ (X[:,k+4:] == i).sum(axis = 1)) * w[:,k]
    for i in range(3):   
        a[i] = a_m[:,:,i].sum()/ a_m.sum()
        b[i] = b_m[:,:,i].sum()/ b_m.sum()
        c[i] = c_m[:,:,i].sum()/ c_m.sum()
        
    theta_new = np.concatenate((a,b,c))
        
    diff = LA.norm(theta_new-theta_old)/LA.norm(theta_old)
    
    return a,b,c,p_new,diff

In [151]:
# Set the tolerance
tol = math.pow(10,-8)

# EM iteration
for i in range(1000):
    print(i)
    w = compute_w(X, a, b, c, p)
    a,b,c,p,diff = update_parameters(X, w,a,b,c,p)
    if diff < tol:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [152]:
# The estimated parameters
a,b,c,p,diff

(array([0.02938031, 0.00561969, 0.965     ]),
 array([1.50746086e-01, 8.49253914e-01, 1.68200290e-20]),
 array([9.07484201e-01, 9.25157992e-02, 3.66066233e-29]),
 array([0.27      , 0.15      , 0.05911207, 0.08088793, 0.09      ,
        0.05      , 0.06      , 0.01      , 0.03010818, 0.04981944,
        0.02007238, 0.03      , 0.00964855, 0.01035145, 0.02007223,
        0.04992777, 0.01      ]),
 2.7417217327211868e-09)